PPCA lavora con più variabili osservabili anzichè una sola

In [1]:
import aesara
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import aesara.tensor as at
import arviz as az
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from scipy import stats
import scipy

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
scaler = StandardScaler()

In [3]:
data = np.load('prova.npy', allow_pickle=True).item()

# extract data social anxiety

In [4]:
import pandas as pd

In [5]:
#csv_ = '/Users/marcoghezzi/PycharmProjects/pythonProject/osfstorage-archive/behavior/LookAtMe_002.csv'
csv_ = '/home/paolo/matteo/matteo/unimi/tesi_master/code/osfstorage-archive/behavior/LookAtMe_002.csv'
global_data = pd.read_csv(csv_, sep='\t')
y = np.array(list([int(d>2) for d in global_data['rating']]))
e_labels = y[:,np.newaxis]  # rating > 2

In [6]:
e_labels = e_labels[1]

e_labels

array([1])

In [7]:
N_e = 1
D_e = 1 #e_labels.shape[1]
print(N_e,D_e)

1 1


## test with new data

In [8]:
import deepemogp.feature_extractor as feature_extractor
import deepemogp.signal.physio as physio
import deepemogp.signal.face as face
import deepemogp.signal.annotation as annotation
import deepemogp.datasets as datasets
import deepemogp.signal.behavior as behavior

# from: https://github.com/SheffieldML/GPy
#import GPy
# from: https://github.com/SheffieldML/PyDeepGP
#import deepgp

import pandas as pd
import numpy as np

show = False
# definition of the feature extractors to be used later
f2 = feature_extractor.FE('wavelet', window=(2, 1))
f3 = feature_extractor.FE('mean', window=(1,0))

# definition of the physiological signals to be extracted
eda_ = physio.EDA(f2)
hr_ = physio.HR(f2)
pupil_ = behavior.PUPIL(f3)

# definition of the emotional annotation to be extracted
#va = annotation.VA('valence', f3)
#ar = annotation.VA('arousal', f3)
# extraction of the desired data from the dataset
d = datasets.FEAR(signals={hr_,pupil_,eda_}, subjects={'2_69'})

for s in d.signals:
    # preprocess ...
    if s.name =='EDA':
        s.preprocess(show=show,new_fps=500)
        s.feature_ext.extract_feat(s,show=show)
    else:
        s.feature_ext.extract_feat_without_preprocess(s, show=show)

    #add feature extraction for eda before preprocessing

    # ... and extract features from each signal type

for sig in d.signals:
    if sig.name=='EDA':
        eda_data = sig.features
    if sig.name=='HR':
        hr_data = sig.features
    if sig.name=='PUPIL':
        pupil_data = sig.features

>> Loading EDA for subject 2 and session 69 from dataset FEAR
>> Loading PUPIL for subject 2 and session 69 from dataset FEAR
>> Loading HR for subject 2 and session 69 from dataset FEAR
>> Processing EDA ...
>> Extracting wavelet features from EDA signal, adopting (2, 1) window ...
>> Extracting mean features from PUPIL signal, adopting (1, 0) window ...
>> Extracting wavelet features from HR signal, adopting (2, 1) window ...


In [9]:
for sig in d.signals:
    print (sig.name)
    print (sig.features)

EDA
         0         1         2         3         4         5         6   \
0  0.835337  0.842847  0.701438  1.224843  2.013294  2.060027  2.012744   
1  1.900922  1.900908  1.901418  1.899583  1.789977  1.644127  1.494651   
2  1.337709  1.337709  1.338517  1.335170  1.222914  1.087731  0.994738   
3  0.940151  0.940143  0.940370  0.939580  0.887408  0.816829  0.743776   
4  0.666393  0.666393  0.666939  0.671945  0.572946  0.717519  2.110283   

         7         8         9         10         11  
0  1.909817  1.776734  1.630724  1.478122   1.344357  
1  1.348431  1.210383  1.075916  0.988751   0.942798  
2  0.944296  0.881046  0.810301  0.735566   0.669671  
3  0.680499  0.555285  0.781479  2.353347   4.587429  
4  4.517740  7.183908  9.054890  9.889524  10.041821  
PUPIL
          0
0  2.185375
1  2.118858
2  2.136183
3  2.079833
4  2.187946
HR
         0         1         2         3         4         5         6   \
0 -1.214719 -0.969489 -1.794818 -0.130028  0.648658 -1.2781

In [10]:
list(d.signals)[0].name
d.signals

{<deepemogp.signal.physio.eda.EDA at 0x7fab7f453e50>,
 <deepemogp.signal.behavior.pupil.PUPIL at 0x7fab761427c0>}

In [11]:
hr_data

,0,1,2,3,4,5,6,7,8,9,10,11
0,-1.214719,-0.969489,-1.794818,-0.130028,0.648658,-1.278176,0.655360,-0.065799,-0.345131,0.717082,-0.309098,-0.737570
1,2.978765,3.028443,3.454952,1.468468,-0.603029,0.742600,-0.289749,-0.124492,0.737946,-0.706377,0.599640,-1.646102
2,-0.786910,-0.700728,-0.907563,-0.142147,0.700752,-0.470463,-0.046469,0.712655,-0.892832,-0.011727,0.577033,-1.191729
3,-1.539465,-1.560444,-1.752965,-0.468737,-0.789217,-0.077583,0.669867,-1.193513,0.529042,0.386148,-0.665355,-1.387225
4,-1.208558,-1.196978,-1.151932,-1.304946,0.252191,0.745649,-1.251371,0.803481,0.048910,-0.673520,0.016190,2.805684


In [12]:
pupil_data

,0
0,2.185375
1,2.118858
2,2.136183
3,2.079833
4,2.187946


In [13]:
eda_data

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.835337,0.842847,0.701438,1.224843,2.013294,2.060027,2.012744,1.909817,1.776734,1.630724,1.478122,1.344357
1,1.900922,1.900908,1.901418,1.899583,1.789977,1.644127,1.494651,1.348431,1.210383,1.075916,0.988751,0.942798
2,1.337709,1.337709,1.338517,1.335170,1.222914,1.087731,0.994738,0.944296,0.881046,0.810301,0.735566,0.669671
3,0.940151,0.940143,0.940370,0.939580,0.887408,0.816829,0.743776,0.680499,0.555285,0.781479,2.353347,4.587429
4,0.666393,0.666393,0.666939,0.671945,0.572946,0.717519,2.110283,4.517740,7.183908,9.054890,9.889524,10.041821


## end test

prendo un'implementazione di sppca e la applico al nostro caso

different types of observable data

1) gaze
2) fisio
    2.1) heart rate variability
    2.2) eda phasic value
3) social anxiety
4) aspettativa del dolore

In [14]:
pupil = pupil_data
hr = hr_data
eda = eda_data
'''social_anx = data['social anxiety'][:,np.newaxis]
pain_exp = data['pain expectation'][:,np.newaxis]'''


"social_anx = data['social anxiety'][:,np.newaxis]\npain_exp = data['pain expectation'][:,np.newaxis]"

In [15]:
print(pupil.shape)
print(hr.shape)
print(eda.shape)

(5, 1)
(5, 12)
(5, 12)


In [16]:
N_pupil = pupil.shape[0]
D_pupil = pupil.shape[1]

N_hr = hr.shape[0]
D_hr = hr.shape[1]

N_eda = eda.shape[0]
D_eda = eda.shape[1]
K = 1

print(N_pupil,D_pupil)
print(N_hr,D_hr)
print(N_eda,D_eda)

5 1
5 12
5 12


In [17]:
with pm.Model() as sPPCA:
    #dati osservabili
    hr_data = pm.MutableData("hr_data", hr.T)
    pupil_data = pm.MutableData("pupil_data", pupil.T)
    eda_data = pm.MutableData("eda_data", eda.T)

    e_data = pm.ConstantData("e_data", e_labels.T)

    #matrici pesi
    Whr = pm.Normal('Whr', mu=at.zeros([D_hr, K]), sigma=2.0 * at.ones([D_hr, K]), shape=[D_hr, K])
    Wpupil = pm.Normal('Wpupil', mu=at.zeros([D_pupil, K]), sigma=2.0 * at.ones([D_pupil, K]), shape=[D_pupil, K])

    Weda = pm.Normal('Weda', mu=at.zeros([D_eda, K]), sigma=2.0 * at.ones([D_eda, K]), shape=[D_eda, K])

    #weight matrix for pain expectation.
    #check mu,sigma,shape
    We = pm.Normal('W_e', mu=at.zeros([D_e, K]), sigma=2.0 * at.ones([D_e,K]), shape=[D_e, K])

    #latent space
    c = pm.Normal('c', mu=at.zeros([N_hr,K]), sigma=at.ones([N_hr,K]), shape=[N_hr,K])

    # dati dell'hrv interpretati come una gaussiana
    x_hr = pm.Normal('x_hr', mu=Whr.dot(c.T), sigma=at.ones([D_hr,N_hr]) ,shape=[D_hr, N_hr], observed=hr_data)
    # dati della dilatazione pupille interpretati come una gaussiana
    x_pupil = pm.Normal('x_pupil', mu=Wpupil.dot(c.T), sigma=at.ones([D_pupil, N_pupil]), shape=[D_pupil, N_pupil], observed=pupil_data)
    #eda
    x_eda = pm.Normal('x_eda',mu=Weda.dot(c.T),sigma= at.ones([D_eda,N_pupil]), shape=[D_eda,N_eda], observed=eda_data)

    # pain expectation. ciò che dovremmo inferire dato c
    # due strade: binary o multiclass (1-4)
    # p = probability of success?
    x_e = pm.Bernoulli('x_e' , p=pm.math.sigmoid(We.dot(c.T)) , shape =[D_e], observed=e_data)

    #x_hr = pm.Bernoulli('x_hr', p=pm.math.sigmoid(Whr.dot(c.T)), shape=[D_hr, N_hr], observed=hr_data)
    #x_eda = pm.Bernoulli('x_eda', p=pm.math.sigmoid(Weda.dot(c.T)), shape=[D_eda, N_eda], observed=eda_data)

In [18]:
gv = pm.model_to_graphviz(sPPCA)
gv.view()

'.gv.pdf'

In [19]:
sPPCA.free_RVs

[Whr ~ N(f(), f()),
 Wpupil ~ N(f(), f()),
 Weda ~ N(f(), f()),
 W_e ~ N(f(), f()),
 c ~ N(f(), f())]

In [ ]:
with sPPCA:
    trace = pm.sample(1000,init='advi+adapt_diag',chains=1,progressbar=True,target_accept=0.95)

qt5ct: using qt5ct plugin
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/16/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/16@2x/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/16/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/16@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/22/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/22@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/24/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/24@2x/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/24/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/24@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/32/"
Invalid Context= "Apps" line

Convergence achieved at 24100
Interrupted at 24,099 [12%]: Average Loss = 354.55
Sequential sampling (1 chains in 1 job)
NUTS: [Whr, Wpupil, Weda, W_e, c]


In [ ]:
az.plot_posterior(trace)

In [ ]:
#az.plot_posterior(mean_field.sample(1000), color="LightSeaGreen")
az.plot_trace(trace)
plt.show()

In [ ]:
az.plot_forest(trace, combined=True, hdi_prob=0.95, r_hat=True);

In [ ]:
print(pupil_data,hr_data,eda_data)

In [ ]:
with sPPCA:
    # update values of predictors:
    pm.set_data(new_data={"pupil_data": pupil,"hr_data":hr,"eda_data":eda})
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123, progressbar=True, var_names=['x_e'])

    eda_pred = posterior_predictive.posterior_predictive["x_e"]

In [ ]:
pupil

In [ ]:
eda_pred[0].plot()

In [ ]:
print(eda_pred[0].shape)

In [ ]:
print(float(np.mean(hr_pred[0])))

median abs deviation prb is not the optimal solution

In [ ]:
hr_pred_mean = (stats.median_abs_deviation(hr_pred[0]))[0]
hr_pred_mean

In [ ]:
hr[0]

In [ ]:
#hr_pred_mode= pd.Series([0.006,0.01,0.02,0.003])

hr and hr_pred_mode devono avere same shape

In [ ]:
train_accuracy_hr = (np.subtract(hr[0], hr_pred_mean))
print('\n\t avg distance hr: \n' + str(train_accuracy_hr))
#print(' ')

In [ ]:
with sPPCA:
    # update values of predictors:
    pm.set_data({"hr_data": hr})
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123)
    gaze_pred = posterior_predictive.posterior_predictive["x_gaze"]

In [ ]:
gaze_pred

In [ ]:
gaze_pred_mean = np.mean(gaze_pred, axis=0).T.reshape(-1)
gaze_true = pupil.reshape(-1)
reconstructed_r2 = metrics.r2_score(gaze_true, gaze_pred_mean)

print('\n\tReconstructed X R2 score: ' + str(reconstructed_r2))

In [ ]:
import deepemogp.feature_extractor as feature_extractor

In [ ]:
f = feature_extractor.FE('wavelet', window=(8,6), params={'w_mother':'db3','w_maxlev':1})

In [ ]:
x = [3, 7, 1, 1, -2, 5, 4, 6]

In [ ]:
f.apply(x)